In [228]:
########## Block 1 ############## <-- Please refer this block number when you ask questions
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set("notebook")

# a useful plotting function
def hist_1d_2d(X, Y, nameX, nameY):
    left, width = 0.1, 0.75
    bottom, height = 0.1, 0.75
    spacing = 0.005
    rect_scatter = [left, bottom, width, height]
    rect_histx = [left, bottom + height + spacing, width, 0.15]
    rect_histy = [left + width + spacing, bottom, 0.15, height]

    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_axes(rect_scatter)
    ax1 = fig.add_axes(rect_histx, sharex=ax)
    ax2 = fig.add_axes(rect_histy, sharey=ax)
    ax1.tick_params(axis="x", labelbottom=False)
    ax2.tick_params(axis="y", labelleft=False)

    ax.scatter(X, Y)
    ax1.hist(X, density=True)
    ax2.hist(Y, orientation='horizontal', density=True)
    ax.set_xlabel(nameX)
    ax.set_ylabel(nameY)

# Gaussian Process Example (30 mins)


## Gaussian Process (unconditioned)

An uncondititoned Gaussian Process can be viewed as a random function $G(x)$, specified by the mean (one-point function)
$$\langle G(x)\rangle \equiv \mu(x),$$ 
and covariance function (two-point function)
$$\langle\delta G(x) \delta G(x')\rangle \equiv k(x, x').$$
And we assume any combinations of the random variable $\{ G(x_1), G(x_2), \cdots, G(x_n)\}$ forms an $n$-dimensional Normal distribution with the above mean and pairwise covaraince.

**Let's examine a conceret example**

### Single and multi-variated Gaussian distribution

1. A single-variable Gaussian distribution is charactered by a mean $\mu$ and a standard deviation $\sigma$

$$f(x; \mu, \sigma) = \frac{1}{(2\pi)^{1/2}\sigma} e^{-\frac{(x-\mu)^2}{2\sigma^2}} $$

2. A multi-varaite Gaussian distribution is charactered by the means $\mu(x_1), \mu(x_2),\cdots$ of each variable and the covariance matrix:
$$ \Sigma = \begin{bmatrix} s_{11} & s_{12} & \cdots\\
s_{21} & s_{22} & \cdots\\ 
\cdots & \cdots & \cdots
\end{bmatrix}
$$
with $\sigma_{12}^2 <\sigma_{11} \sigma_{22}$


$$f(x_1, x_2; \mu_1, \mu_2, \Sigma_{ij}) = \frac{1}{(2\pi)^{1/2}\sqrt{|\Sigma|}} \exp\left\{-\frac{1}{2}\sum_{i,j=1}^2(x_i-\mu_i)^T \Sigma_{ij}^{-1} (x_j-\mu_j)\right\} $$

We will take a specific form of the kernel function 
$$k(x_1, x_2) = C^2 \exp\left\{-\frac{|x_1-x_2|^2}{2L^2}\right\}$$
**Output values of close input points $x_{1,2}$ have strongly correlated / close outputs $y_{1,2}$**

In [ ]:
########## Block 2 ##############
@np.vectorize
def kernel(x1, x2, C, L):
    return C**2 * np.exp(-.5*(x1-x2)**2/L**2)
def ND_Normal(N=2, C=1, L=1, nsamples=1000):
    x = np.linspace(-1,1,N)
    mean = np.zeros_like(x)
    cov = kernel(*np.meshgrid(x,x), C, L)
    return x, np.random.multivariate_normal(mean, cov, nsamples)

x, Y = ND_Normal(N=10, C=1, L=1)
hist_1d_2d(Y[:,0], Y[:,1], '$x_1$', '$x_2$')

Another way to look at this multi-variate normal, plot $x$ and realizations $y$ as a scattered plot. Since close inputs gaurantees close outputs, the resulting plots are samples of smooth functions with the given variance and correlation function.

In [ ]:
########## Block 3 ##############
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,4))
_ = ax1.plot(x, Y[:20].T, 'ro-')
ax1.set_title("20 realizations of the rasndom function")

CLbins = [0,60,90,95]
ax1.set_title("Probability distribution of values of $G(x)$")
for CL, opacity in zip(CLbins, [1., .4, .3, .2, .1]):
    lower, upper = np.percentile(Y, [50-CL/2., 50+CL/2.], axis=0)
    ax2.fill_between(x, lower, upper, color='r', alpha=opacity)

### GP-1: Conditioning the GP
Now, we would like to pick a particular subset of random functions that comes near to the points
$$(x^*_i, y^*_i) =  (-1,-1), (0,0.5),(1,0.7)$$.
To do this, we picks random functions that statisfies $$|G(x^*_i)-y^*_i | < \epsilon$$

In [ ]:
########## Block 4 ##############
x, Y = ND_Normal(N=11, C=1, L=1, nsamples=100000)
epsilon = 0.1
cut = ( np.abs(Y[:,0]+1)<epsilon ) \
    & ( np.abs(Y[:,5]-.5)<epsilon ) \
    & ( np.abs(Y[:,10]-.7)<epsilon ) \

fig, ax = plt.subplots(1,1, figsize=(4,4))
CLbins = [0,60,90,95]
ax.set_title("Probability distribution of values of $G(x)$")
for CL, opacity in zip(CLbins, [1., .4, .3, .2, .1]):
    lower, upper = np.percentile(Y, [50-CL/2., 50+CL/2.], axis=0)
    ax.fill_between(x, lower, upper, color='r', alpha=opacity)
ax.errorbar([-1,0,1],[-1,.5,.7],yerr=epsilon,fmt='kD')
_ = ax.plot(x, Y[cut].T,'b-')

The conditioned set of random functions provides an interpolation of the three conditioned points with their spread as uncertainties. The natural inclusion of interpolation uncertainty is a big advantage of GP in the workflow of Bayesian analysis of complex model.

### The variance and the correlations length $C^2, L$.
Strickly spearking, the variance and the correlation length is unknown for a given set of data to be interpolated. An common practice is to optimize the values of $C$ and $L$ in the "fitting" so that it maximumize the likelihood of desribing the data. The systematic tuning for an optimal set of $C$ and $L$ is the so-called training process.

For practical use, we can use well developed GP modules in sklearn. It implements different kinds of kernel functions and training algorithms.

https://scikit-learn.org/stable/tutorial/basic/tutorial.html

https://scikit-learn.org/stable/modules/gaussian_process.html

## Applying sklearn GP to 1D inference/interpolation

**The problem**: given values of function $F(x)$ on a sparse grid $x_i, i=1,2,\cdots$, use Gaussain process emulator (regressor) to infer the functional form of the $F$.

$$F(x) = x^2 + \sin(5x)$$

In [ ]:
########## Block 5 ##############
# This is function to be emualted
def F(x):
    return x**2+np.sin(5*x)

x_design = np.linspace(-1,1,6)
y_design = F(x_design)
plt.plot(x_design, y_design, 'ro', label='Design')
x = np.linspace(-1,1,101)
plt.plot(x, F(x),'k-', label=r'$F(x)$')
plt.xlabel(r"$x$")
plt.ylabel(r"$y$")

In [ ]:
########## Block 6 ##############
from sklearn.gaussian_process import \
     GaussianProcessRegressor as GPR
from sklearn.gaussian_process import kernels

# Train the emulator, specify the kernel functions 
kernel = \
    1 * kernels.RBF(
        length_scale=1.,
        length_scale_bounds=(.1,10)
    )

gp = GPR(kernel=kernel, n_restarts_optimizer=5)
gp.fit(np.atleast_2d(x_design).T, y_design)
print("C^2 = ", gp.kernel_.get_params()['k1'])
print(gp.kernel_.get_params()['k2'])
print("This score of describing the training data:", gp.score(np.atleast_2d(x_design).T, y_design))

xv = np.linspace(-1,1,11)
print("This score of describing validation data:", gp.score(np.atleast_2d(xx).T, F(xv)) )


In [234]:
########## Block 7 ##############
# A wrapper to make predictions from GP
def predict(x, gp):
    mean, cov = gp.predict(return_cov=True, X=np.atleast_2d(x).T)
    return mean, np.sqrt(np.diag(cov))

In [ ]:
########## Block 8 ##############
x = np.linspace(-1,1,101)
y, ystd = predict(x, gp)

plt.plot(x_design, y_design, 'ro', label='Design')
plt.plot(x, F(x),'k-', label=r'$F(x)$')
plt.plot(x, y,'b--', label=r'GP mean')
plt.fill_between(x, y-ystd, y+ystd, color='b', alpha=.3, label=r'GP $\pm 1\sigma$')
plt.fill_between(x, y-2*ystd, y+2*ystd, color='gray', alpha=.3, label=r'GP $\pm 2\sigma$')
plt.xlabel(r"Input $x$")
plt.ylabel(r"Output $y=F(x)$")
plt.legend()

    Go back to make the prediction range of $x$ larger, `[-2,2]` for instance. Does this Gaussian provide good extrapolation? (However, it is possible to use GP to make projections)